In [2]:
import pandas as pd
import numpy as np
import os
import joblib
import time
import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

In [3]:
CATEGORY_NAME = 'Toys_and_Games'
TEST_SIZE = 0.20
RANDOM_STATE = 42
CV_FOLDS = 3
N_JOBS = 4

In [4]:
base_path = os.path.dirname(os.getcwd())
data_dir = os.path.join(base_path, "data", "processed", CATEGORY_NAME)
data_path = os.path.join(data_dir, f"{CATEGORY_NAME.lower()}.parquet")

In [5]:
data = pd.read_parquet(data_path)
X = data.drop("class", axis=1)
y = data["class"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

In [7]:
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
preprocessor = ColumnTransformer(transformers=[
    ('text', TfidfVectorizer(), 'cleaned_text'),
    ('numeric', numeric_transformer, ['overall', 'helpfulness_ratio'])
])

In [8]:
pipeline_lgbm = Pipeline([
    ('preprocessor', preprocessor),
    ('sampler', RandomUnderSampler(random_state=RANDOM_STATE)),
    ('model', lgb.LGBMClassifier(random_state=RANDOM_STATE))
])

In [9]:
param_grid_lgbm = {
    'preprocessor__text__max_features': [15000], # Önceki deneyden en iyi sonucu aldığımız değer
    'model__n_estimators': [100, 200], # Ağaç sayısı
    'model__learning_rate': [0.1, 0.05], # Öğrenme oranı
    'model__num_leaves': [31, 50] # Her ağacın karmaşıklığı
}

In [ ]:
grid_search_lgbm = GridSearchCV(pipeline_lgbm, param_grid_lgbm, cv=CV_FOLDS, scoring='f1_weighted', n_jobs=N_JOBS, verbose=2)
print(f"GridSearchCV for LightGBM starting.")
start_time = time.time()
grid_search_lgbm.fit(X_train, y_train)
end_time = time.time()
duration_minutes = (end_time - start_time) / 60

LightGBM için GridSearchCV başlıyor... (Bu işlem Naive Bayes'ten daha uzun sürecektir)
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[LightGBM] [Info] Number of positive: 267509, number of negative: 267509
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 9.470616 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 738018
[LightGBM] [Info] Number of data points in the train set: 535018, number of used features: 14732
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

In [11]:
print("\n" + "="*50)
print("LGBM HİPERPARAMETRE OPTİMİZASYONU SONUÇLARI")
print("="*50)
print(f"Toplam Süre: {duration_minutes:.2f} dakika")
print(f"En iyi F1 Skoru (Çapraz Doğrulama ile): {grid_search_lgbm.best_score_:.4f}")
print("Bulunan En İyi Parametreler:")
print(grid_search_lgbm.best_params_)
print("-" * 50)
best_lgbm_model = grid_search_lgbm.best_estimator_


LGBM HİPERPARAMETRE OPTİMİZASYONU SONUÇLARI
Toplam Süre: 12.84 dakika
En iyi F1 Skoru (Çapraz Doğrulama ile): 1.0000
Bulunan En İyi Parametreler:
{'model__learning_rate': 0.1, 'model__n_estimators': 100, 'model__num_leaves': 31, 'preprocessor__text__max_features': 15000}
--------------------------------------------------


In [12]:
results_file = os.path.join(base_path, "reports", "model_results_log.csv")
report = classification_report(y_test, best_lgbm_model.predict(X_test), output_dict=True)
result_data = {
    'category': f"{CATEGORY_NAME}_LGBM", # Model adını belirtmek için
    'best_cv_f1_score': grid_search_lgbm.best_score_,
    'test_accuracy': report['accuracy'],
    'test_f1_real_review': report['0']['f1-score'],
    'test_precision_real_review': report['0']['precision'],
    # ... (diğer metrikler)
    'best_params': str(grid_search_lgbm.best_params_),
    'training_time_minutes': duration_minutes
}
temp_df = pd.DataFrame([result_data])
header = not os.path.exists(results_file)
temp_df.to_csv(results_file, mode='a', header=header, index=False)
print(f"LGBM sonuçları '{results_file}' dosyasına eklendi.")

c:\Users\Aybars\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LGBM sonuçları 'c:\work environment\Projects\amazon-spam-review\reports\model_results_log.csv' dosyasına eklendi.


In [13]:
model_dir = os.path.join(base_path, "models", CATEGORY_NAME)
os.makedirs(model_dir, exist_ok=True)
model_filename = f"lightgbm_{CATEGORY_NAME.lower()}.joblib"
model_path = os.path.join(model_dir, model_filename)
joblib.dump(best_lgbm_model, model_path)
print(f"Eğitilmiş LightGBM modeli '{model_path}' olarak kaydedildi.")

Eğitilmiş LightGBM modeli 'c:\work environment\Projects\amazon-spam-review\models\Toys_and_Games\lightgbm_toys_and_games.joblib' olarak kaydedildi.


The lightGBM model has 1.0 accuracy, thus this creates a suspition that this dataset is artificial and to test that, here is some results about lightGBM model:

In [14]:
import pandas as pd
import numpy as np

# 'best_lgbm_model' değişkeninin bir önceki koddan yüklü olduğunu varsayıyorum
# Eğer değilse, önce modeli yükle:
# import joblib
# best_lgbm_model = joblib.load('path/to/your/lightgbm.joblib')

# Pipeline'dan son adımı, yani asıl LGBM modelini al
final_lgbm_model = best_lgbm_model.named_steps['model']

# Pipeline'dan ön işlemciyi ve onun içindeki TfidfVectorizer'ı al
preprocessor = best_lgbm_model.named_steps['preprocessor']
vectorizer = preprocessor.named_transformers_['text']

# TfidfVectorizer'ın öğrendiği tüm kelimelerin (özelliklerin) listesini al
text_feature_names = vectorizer.get_feature_names_out()

# Sayısal özelliklerin adlarını da ekle
numeric_feature_names = ['overall', 'helpfulness_ratio']
all_feature_names = np.concatenate([text_feature_names, numeric_feature_names])

# Özellik önemlerini bir DataFrame'e aktar
feature_importances = pd.DataFrame({
    'feature': all_feature_names,
    'importance': final_lgbm_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Modelin En Önemli Gördüğü 20 Özellik:")
print(feature_importances.head(20))

Modelin En Önemli Gördüğü 20 Özellik:
          feature  importance
14             aa         234
15000     overall         100
25        ability          29
1054      battery          19
5193          fun          11
27           able           9
388          also           9
202         adult           8
131      activity           8
4352         even           8
70      accessory           8
5536         good           6
7493         like           6
23            abc           5
1464       bought           5
525        anyway           4
5670        great           4
15            aaa           3
631           arm           3
37     absolutely           3


In [15]:
# 'data' DataFrame'inin yüklü olduğunu varsayıyorum

# Veriyi ikiye ayır
real_reviews_text = data[data['class'] == 0]['cleaned_text']
spam_reviews_text = data[data['class'] == 1]['cleaned_text']

# Her sınıf için eşsiz kelimelerden oluşan bir set oluştur
real_words = set(" ".join(real_reviews_text).split())
spam_words = set(" ".join(spam_reviews_text).split())

# Kümeleri kullanarak farkları bul
spam_only_words = spam_words - real_words
real_only_words = real_words - spam_words

print(f"\nSadece SPAM yorumlarda bulunan eşsiz kelime sayısı: {len(spam_only_words)}")
print(f"Sadece GERÇEK yorumlarda bulunan eşsiz kelime sayısı: {len(real_only_words)}")

# Birkaç örnek gösterelim
print("\nSadece SPAM'de geçen bazı kelimeler:", list(spam_only_words)[:20])


Sadece SPAM yorumlarda bulunan eşsiz kelime sayısı: 582252
Sadece GERÇEK yorumlarda bulunan eşsiz kelime sayısı: 119616

Sadece SPAM'de geçen bazı kelimeler: ['lavenderitem', 'eventthis', 'oowashi', 'afterparty', 'areumx', 'belthighly', 'containerbetter', 'includedtook', 'directbuy', 'hfisher', 'motorboth', 'monstertail', 'mendenhall', 'gamesduring', 'happypleasecontinue', 'breaksits', 'magicianintraining', 'fabuleux', 'cxx', 'flooro']


Thus, this dataset is not trustable. I will move on to a more reliable dataset